In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import os
import pymysql
from tqdm import tqdm

In [4]:
ip_address = os.environ['AWS_IP']
PASSWORD = os.environ['SQL_PWD_AWS']

con = pymysql.connect(
    user='edward', 
    passwd=PASSWORD,
    host=ip_address, 
    db='community', 
    charset='utf8'
)

In [7]:
BASE_URL = 'https://pann.nate.com'

In [50]:
def continuos_dates(start, end, fm):
    dates = []
    dt = datetime.datetime.strptime(start, fm)
    while True:
        dt_string = dt.strftime(format=fm)
        dates.append(dt_string)
        if dt_string == end:
            break
        dt = dt + datetime.timedelta(days=1)
    return dates

In [42]:
def get_urls(date):
    # %Y-%m-%d
    sql = f"select * from urls where date = '{date}'"
    with con.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
    return result

In [62]:
def is_dup_content(article_id):
    sql = f"select * from nate_content where id = {article_id}"
    with  con.cursor() as cursor:
        cursor.execute(sql)
#     return bool(cursor.fetchall())
    return cursor.fetchall()

In [9]:
def commit_content(values):
    sql = f'insert into nate_content(id, title, content, date) values {values}'
    with  con.cursor() as cursor:
        cursor.execute(sql)
        con.commit()

In [10]:
def commit_comment(values):
    sql = f'insert into nate_comment(good, bad, content, article_id) values {values}'
    with  con.cursor() as cursor:
        cursor.execute(sql)
        con.commit()

In [54]:
dates = continuos_dates('2021-01-02', '2021-01-10', '%Y-%m-%d')

In [57]:
urls[0]

(901, 'nate_pann', '/talk/356809550', datetime.date(2021, 1, 10))

In [55]:
for date in dates:
    print(date)
    urls = get_urls(date)
    for u in tqdm(urls):
        # 글 내용 저장
        article_id, kind, url_tail, date = u
        if is_dup_content(article_id):  # 이미 db에 저장되어있으면
            continue
        url = BASE_URL + url_tail
        soup = BeautifulSoup(requests.get(url).text, 'lxml')
        title = soup.find('title').text
        content = soup.find('div', {'id': 'contentArea'}).text.strip()
        values = (article_id, title, content, date.strftime('%Y-%m-%d'))
        commit_content(values)

        # 댓글 저장
        comments = soup.find('div', {'class':'commentBox'}).find_all('dl', {'class': 'cmt_item'})
        for comment in comments:
            if len(comment.find_all('dt')) <= 1:  # 삭제된 댓글
                continue

            good = int(comment.find('dd', {'class': 'n_good'}).text) 
            bad = int(comment.find('dd', {'class': 'n_bad'}).text) 
            cm = comment.find('dd', {'class': 'usertxt'}).text.strip()
            values = (good, bad, cm, article_id)
            commit_comment(values)

  0%|          | 0/100 [00:00<?, ?it/s]

2021-01-02


  0%|          | 0/100 [00:00<?, ?it/s]

2021-01-03


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

2021-01-09


  0%|          | 0/100 [00:00<?, ?it/s]

2021-01-10


100%|██████████| 100/100 [01:15<00:00,  1.33it/s]
